Pycaret
https://pycaret.gitbook.io/docs/get-started/tutorials
https://github.com/pycaret/pycaret/blob/master/tutorials/time_series/forecasting/univariate_without_exogeneous_part1.ipynb

Algorithm Trading
https://www.youtube.com/watch?v=FEDBsbTFG1o
https://www.youtube.com/watch?v=PuZY9q-aKLw

TODO:
Achar alguns datasets melhores de timeseries (Rami Krispin)

In [33]:
import pandas as pd
import matplotlib.pyplot as plt
#import pandas_datareader as web
from pandas_datareader import data as web
import datetime as dt
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Obtendo Bitcoin to USD

In [ ]:
crypto='BTC'
against='USD'

start=dt.datetime(2016,1,1)
end=dt.datetime.now()

import yfinance as yfin
yfin.pdr_override()
#bit_usd = web.DataReader(teste,'yahoo',start,end)
data = yfin.download(f'{crypto}-{against}', start , end)
data.head()

# Obtendo cotação BRL-USD

In [ ]:
import yfinance as yf

# Define the ticker symbol for the BRL to USD exchange rate
ticker = 'BRL=X'

# Set the start and end dates for the data
start_date = '2016-01-01'
end_date = dt.datetime.now()

# Retrieve the data from Yahoo Finance
data = yf.download(ticker, start=start_date, end=end_date)

data.reset_index()

# Print the first few rows of the DataFrame
print(data.head())

In [ ]:
time_periods = [7, 14, 30, 60]
for period in time_periods:
    ma_col = 'MA{}'.format(period)
    data[ma_col] = data['Close'].rolling(period).mean()

#data['MA12'] = data['Close'].rolling(12).mean()
data.tail()

In [ ]:
import plotly.express as px
fig = px.line(data.reset_index(), x="Date", y=["Close", "MA7","MA14","MA30","MA60"], template = 'plotly_dark')
fig.show()

# Dados de consumo de Gás US

In [ ]:
data = pd.read_csv("dados.csv",index_col=False)
data = data.loc[:,['date','y']]
data['date'] = pd.to_datetime(data['date'])
data.head()

In [ ]:
# extract month and year from dates**
data.reset_index(inplace=True)
data['Day'] = [i.day for i in data['date']]
data['Month'] = [i.month for i in data['date']]
data['Year'] = [i.year for i in data['date']]

# create a sequence of numbers
#data['Series'] = np.arange(1,len(data)+1)

# drop unnecessary columns and re-arrange
#data.drop(['Date', 'MA7','MA14','MA30','MA60'], axis=1, inplace=True)
#data = data[['Series', 'Year', 'Month', 'Day','Close']] 
data = data[[ 'Year', 'Month', 'Day','y']] 

# check the head of the dataset**
data.head()

# Preparando dados

# EDA

In [ ]:
from pycaret.time_series import TSForecastingExperiment
eda = TSForecastingExperiment()
fig_kwargs = {
    # "renderer": "notebook",
    "renderer": "png",
    "width": 1200,
    "height": 800,
}
eda.setup(data=data, fh=21,fig_kwargs=fig_kwargs,target='y')

In [ ]:
eda.plot_model()
#eda.plot_model(plot="periodogram")
#eda.plot_model(plot="fft")
#eda.plot_model(plot="acf")
eda.plot_model(plot="diagnostics")

In [ ]:
eda.check_stats()
eda.check_stats(test="summary")
eda.plot_model(plot="train_test_split", fig_kwargs={"height": 400, "width": 900})
eda.plot_model(plot="decomp_stl", fig_kwargs={"height": 500})
# Show the Cross Validation splits inside the train set
# The blue dots represent the training data for each fold.
# The orange dots represent the validation data for each fold
eda.plot_model(plot="cv", fig_kwargs={"height": 400, "width": 900})

# Modeling

In [ ]:
train = data[data['Year'] < 2021]
test = data[data['Year'] >= 2021]
train.shape, test.shape

In [ ]:
from pycaret.regression import *
s = setup(data = train, test_data = test, target = 'y', fold_strategy = 'timeseries', numeric_features = ['Year', 'Month','Day'], fold = 5, session_id = 123)

In [ ]:
#best = compare_models(sort='R2')
#best
best_baseline_models = compare_models(n_select=3)
best_baseline_models

In [ ]:
best_tuned_models = [tune_model(model) for model in best_baseline_models]
best_tuned_models

In [ ]:
for i in range(0,3):
    plot_model(best_tuned_models[i], plot='error')
    plot_model(best_tuned_models[i])
    predict_model(best_tuned_models[i])
    print(best_tuned_models[i])

In [ ]:
#lightgbm = create_model('br')
#tuned_lightgbm = tune_model(lightgbm)

In [ ]:
prediction_holdout = predict_model(top5)

In [ ]:
predictions = predict_model(best_tuned_models[0], data=data)
predictions.head()

In [ ]:
import plotly.express as px
predictions['date'] = pd.to_datetime(data[['Day', 'Month', 'Year']])
fig = px.line(predictions, x='date', y=["y", "prediction_label"], template = 'plotly_dark')
fig.add_vrect(x0="2013-01-01", x1="2022-12-31", fillcolor="grey", opacity=0.25, line_width=0)
fig.show()

In [ ]:
final_best = finalize_model(best_tuned_models[0])

In [ ]:
#future_dates = pd.date_range('2023-04-24', '2023-05-20', freq='B')
future_dates = pd.date_range('2023-01-01', '2024-06-01', freq='MS')

future_df = pd.DataFrame()

future_df['Day'] = [i.day for i in future_dates]
future_df['Month'] = [i.month for i in future_dates]
future_df['Year'] = [i.year for i in future_dates]    
#future_df['Series'] = np.arange(28,(28+len(future_dates)))
future_df.head()

In [ ]:
future_df

In [ ]:
predictions_future = predict_model(final_best, data=future_df)
predictions_future.head()

In [ ]:
concat_df = pd.concat([data,predictions_future], axis=0)
df2 = pd.concat([predictions.loc[:,["date"]], pd.DataFrame(future_dates, columns=['date'])], ignore_index=True)

concat_df_i = pd.to_datetime(df2["date"])
concat_df.set_index(concat_df_i, inplace=True)
concat_df.head()
#fig = px.line(concat_df, x=concat_df.index, y=["Close", "prediction_label"], template = 'plotly_dark')
#fig.show()

In [ ]:
fig = px.line(concat_df, x=concat_df.index, y=["y", "prediction_label"], template = 'plotly_dark')
fig.show()

In [ ]:
save_model(final_best, 'teste')

In [ ]:
create_api(final_best, 'api')